# Analyze captions and try to get list of nouns of different images.
* Is object from segmentation in the caption?
* Analyze nouns in captions to see if they correlate with coco object names

## Start with filtered dataset with `person` class

In [3]:
import json
import pickle
import random
import collections
import nltk
import os
nltk.download('averaged_perceptron_tagger')
COCO_PATH = "../../datasets/coco"

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/braian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# load processed file
data_split = "val2017"
# data_split = "train2017"
dataset_filename = 'task1_person_accessory_data_no_overlap.json'
# dataset_filename = 'task5_person_data_no_overlap.json'

with open(f'filtered_datasets/{data_split}/{dataset_filename}', "r") as f:
    filtered_task1_data = json.load(f)
    
# category name map
with open('coco_category_id_map.pickle', 'rb') as handle:
    category_id_map = pickle.load(handle)

In [4]:
# with open(os.path.join(COCO_PATH, "annotations/captions_val2017.json"), 'r') as f:
#     coco_captions = json.load(f)    

In [5]:
filtered_task1_data.keys()

dict_keys(['32', '28', '31', '27'])

In [6]:
for obj_class, annotations in filtered_task1_data.items():
    print(f"{category_id_map[int(obj_class)]}:  {len(annotations)}")

tie:  133
umbrella:  146
handbag:  155
backpack:  188


In [7]:
caption = filtered_task1_data[list(filtered_task1_data.keys())[0]][10]['annotations']['captions']
caption

['A bus that is driving down the road.',
 'A city bus is driving through the town',
 'no image again there are a lot of these lately',
 'A city bus is driving down the road next to buildings.',
 'A black passenger bus navigating paved city streets.']

In [8]:
# import sng_parser
# graph = sng_parser.parse(caption)
# graph

In [9]:
word_tokens = nltk.word_tokenize('A women is riding her bikes down the street in front of traffic.')
print(word_tokens)
tag = nltk.pos_tag(word_tokens)
print(tag)

['A', 'women', 'is', 'riding', 'her', 'bikes', 'down', 'the', 'street', 'in', 'front', 'of', 'traffic', '.']
[('A', 'DT'), ('women', 'NNS'), ('is', 'VBZ'), ('riding', 'VBG'), ('her', 'PRP$'), ('bikes', 'NNS'), ('down', 'RP'), ('the', 'DT'), ('street', 'NN'), ('in', 'IN'), ('front', 'NN'), ('of', 'IN'), ('traffic', 'NN'), ('.', '.')]


In [10]:
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
# Initialize wordnet lemmatizer
wnl = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/braian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/braian/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
nouns = [token for token, pos_tag in tag if pos_tag in ['NN','NNS']]
nouns

['women', 'bikes', 'street', 'front', 'traffic']

In [12]:
lemma_nouns = [wnl.lemmatize(token, pos="n") for token, pos_tag in tag if pos_tag in ['NN','NNS']]
lemma_nouns

['woman', 'bike', 'street', 'front', 'traffic']

In [13]:
[(category_id_map[int(obj_id)], obj_id) for obj_id in filtered_task1_data]

[('tie', '32'), ('umbrella', '28'), ('handbag', '31'), ('backpack', '27')]

### extract all nouns from captions, save as a list 
Nouns for a particular image should match the list of distinct categories in segmentation

In [14]:
word_counts = {}
for obj_class in filtered_task1_data:    
    noun_list = []
    for idx, ann in enumerate(filtered_task1_data[obj_class]):
        captions = ann['annotations']['captions']
        ann['annotations']["caption_nouns"] = []
        for caption in captions:
            tokens = nltk.word_tokenize(caption)
            tag = nltk.pos_tag(tokens)
            list_of_nouns = [wnl.lemmatize(token, pos="n").lower() for token, pos_tag in tag if pos_tag in ['NN', 'NNS']]
            ann['annotations']["caption_nouns"].extend(list_of_nouns)
            noun_list.extend(list_of_nouns)
            if random.random() > 0.9:
                print(f"[{idx, category_id_map[int(obj_class)]}] caption : {caption}")           
                print(f"[{idx, category_id_map[int(obj_class)]}] capt. noun : {sorted(ann['annotations']['caption_nouns'])}")
                print(f"[{idx, category_id_map[int(obj_class)]}] segm. obj  : {sorted([category_id_map[obj_id] for obj_id in ann['annotations']['distinct_obj']])}")
        # make list unique now that I'm using all captions
        ann['annotations']["caption_nouns"] = list(set(ann['annotations']["caption_nouns"]))
    word_counts[obj_class] = collections.Counter(noun_list)

[(1, 'tie')] caption : A man standing in front of a bathroom mirror and a sink.
[(1, 'tie')] capt. noun : ['bathroom', 'front', 'front', 'man', 'man', 'mirror', 'mirror', 'room', 'sink']
[(1, 'tie')] segm. obj  : ['bottle', 'bowl', 'cell phone', 'cup', 'person', 'sink', 'tie', 'toothbrush']
[(5, 'tie')] caption : A man with a red hat is in the city.
[(5, 'tie')] capt. noun : ['blazer', 'buckle', 'city', 'hat', 'hat', 'hat', 'hat', 'man', 'man', 'man', 'man', 'silver', 'suit', 'suit', 'tie', 'tie']
[(5, 'tie')] segm. obj  : ['car', 'person', 'tie']
[(6, 'tie')] caption : A man and woman at a party dressed in costumes.
[(6, 'tie')] capt. noun : ['banana', 'camera', 'costume', 'hat', 'helmet', 'helmet', 'man', 'man', 'party', 'people', 'woman', 'woman']
[(6, 'tie')] segm. obj  : ['banana', 'person', 'tie']
[(6, 'tie')] caption : There are two people posing and one man is holding a banana 
[(6, 'tie')] capt. noun : ['banana', 'banana', 'camera', 'costume', 'hat', 'helmet', 'helmet', 'man',

[(96, 'tie')] caption : An open market has a large table of bananas.
[(96, 'tie')] capt. noun : ['banana', 'banana', 'banana', 'bunches', 'man', 'market', 'market', 'market', 'shopper', 'street', 'table']
[(96, 'tie')] segm. obj  : ['banana', 'car', 'person', 'tie']
[(96, 'tie')] caption : A man looking over a full stand of bananas.
[(96, 'tie')] capt. noun : ['banana', 'banana', 'banana', 'banana', 'bunches', 'man', 'man', 'market', 'market', 'market', 'shopper', 'stand', 'street', 'table']
[(96, 'tie')] segm. obj  : ['banana', 'car', 'person', 'tie']
[(98, 'tie')] caption : A couple cut their wedding cake on their wedding day.
[(98, 'tie')] capt. noun : ['cake', 'cake', 'couple', 'cut', 'day', 'knife', 'man', 'wedding', 'wedding', 'woman']
[(98, 'tie')] segm. obj  : ['cake', 'knife', 'person', 'tie']
[(98, 'tie')] caption : A man and woman making a cut into a cake together
[(98, 'tie')] capt. noun : ['cake', 'cake', 'cake', 'couple', 'cut', 'cut', 'day', 'knife', 'man', 'man', 'weddi

[(48, 'umbrella')] caption : A little girl walking down a driveway carrying a pink umbrella.
[(48, 'umbrella')] capt. noun : ['driveway', 'girl', 'girl', 'pink', 'umbrella', 'umbrella']
[(48, 'umbrella')] segm. obj  : ['car', 'person', 'umbrella']
[(56, 'umbrella')] caption : a mans shadow in front of lighting equipment
[(56, 'umbrella')] capt. noun : ['airbag', 'cast', 'equipment', 'front', 'front', 'front', 'head', 'head', 'man', 'man', 'man', 'man', 'shadow', 'shadow', 'silhouette', 'umbrella', 'umbrella', 'umbrella']
[(56, 'umbrella')] segm. obj  : ['person', 'umbrella']
[(58, 'umbrella')] caption : A beach setting with lots of people and sun umbrellas.
[(58, 'umbrella')] capt. noun : ['beach', 'beach', 'beach', 'beach', 'chair', 'chair', 'lot', 'lounge', 'people', 'people', 'people', 'people', 'umbrella', 'umbrella', 'walking', 'water']
[(58, 'umbrella')] segm. obj  : ['book', 'chair', 'cup', 'person', 'umbrella']
[(63, 'umbrella')] caption : Two women sitting outside,one holding 

[(2, 'handbag')] caption : People are sitting on the ground petting a cat.
[(2, 'handbag')] capt. noun : ['camera', 'cat', 'cat', 'cat', 'cat', 'ground', 'group', 'group', 'group', 'hall', 'people', 'people', 'people', 'people', 'picture', 'woman']
[(2, 'handbag')] segm. obj  : ['cat', 'cell phone', 'handbag', 'person']
[(4, 'handbag')] caption : Urban downtown city center with a bicyclist. and pedestrians 
[(4, 'handbag')] capt. noun : ['bicyclist', 'bike', 'building', 'car', 'center', 'city', 'city', 'group', 'men', 'pedestrian', 'people', 'side', 'street']
[(4, 'handbag')] segm. obj  : ['bicycle', 'car', 'handbag', 'person', 'traffic light']
[(5, 'handbag')] caption : A street area with a building and cars.
[(5, 'handbag')] capt. noun : ['area', 'building', 'car', 'street']
[(5, 'handbag')] segm. obj  : ['bicycle', 'car', 'handbag', 'parking meter', 'person', 'traffic light']
[(7, 'handbag')] caption : People watching a small boat being raised up.
[(7, 'handbag')] capt. noun : ['boa

[(96, 'handbag')] caption : A woman and girl standing with luggage and giving a peace sign.
[(96, 'handbag')] capt. noun : ['bag', 'front', 'girl', 'girl', 'girl', 'girl', 'hand', 'luggage', 'luggage', 'peace', 'peace', 'sign', 'sign', 'suitcase', 'suitcase', 'suitcase', 'woman', 'woman', 'woman']
[(96, 'handbag')] segm. obj  : ['handbag', 'person', 'suitcase']
[(99, 'handbag')] caption : A stack of luggage with an opened one on top.
[(99, 'handbag')] capt. noun : ['brief', 'brown', 'case', 'display', 'group', 'luggage', 'luggage', 'piece', 'purse', 'sidewalk', 'stack', 'stack', 'street', 'suitcase', 'suitcase', 'top', 'woman']
[(99, 'handbag')] segm. obj  : ['handbag', 'person', 'suitcase']
[(100, 'handbag')] caption : Two kids play in the fresh produce section of a busy grocery store.
[(100, 'handbag')] capt. noun : ['grocery', 'grocery', 'kid', 'kid', 'produce', 'produce', 'section', 'section', 'store', 'store']
[(100, 'handbag')] segm. obj  : ['apple', 'dog', 'handbag', 'orange', '

[(37, 'backpack')] caption : A train platform with trains on the other side
[(37, 'backpack')] capt. noun : ['awning', 'passenger', 'passing', 'platform', 'platform', 'side', 'station', 'station', 'train', 'train', 'train', 'train', 'train', 'train']
[(37, 'backpack')] segm. obj  : ['backpack', 'bench', 'person', 'train']
[(39, 'backpack')] caption : Three people are sitting on a bench looking out at mountains and a lake.
[(39, 'backpack')] capt. noun : ['bench', 'bench', 'lake', 'mountain', 'park', 'people', 'water']
[(39, 'backpack')] segm. obj  : ['backpack', 'bench', 'person']
[(39, 'backpack')] caption : Three persons seated on a bench near a pool of water
[(39, 'backpack')] capt. noun : ['bench', 'bench', 'bench', 'bench', 'bench', 'front', 'group', 'lake', 'lake', 'lake', 'mountain', 'park', 'people', 'people', 'people', 'person', 'pool', 'water', 'water']
[(39, 'backpack')] segm. obj  : ['backpack', 'bench', 'person']
[(41, 'backpack')] caption : A man with a bike walks towards

[(135, 'backpack')] caption : A group of people standing next to a double decker bus.
[(135, 'backpack')] capt. noun : ['bus', 'decker', 'group', 'people']
[(135, 'backpack')] segm. obj  : ['backpack', 'bus', 'car', 'handbag', 'person']
[(135, 'backpack')] caption : A group of people standing outside of a double decker bus.
[(135, 'backpack')] capt. noun : ['bus', 'bus', 'bus', 'bus', 'bus', 'curb', 'decker', 'decker', 'group', 'group', 'people', 'people', 'people', 'people', 'side', 'sidewalk', 'sits', 'walk']
[(135, 'backpack')] segm. obj  : ['backpack', 'bus', 'car', 'handbag', 'person']
[(140, 'backpack')] caption : THERE IS A BUS PARKED ON THE SIDE OF THE STREET
[(140, 'backpack')] capt. noun : ['bus', 'crowd', 'stop']
[(140, 'backpack')] segm. obj  : ['backpack', 'bus', 'handbag', 'person', 'train']
[(142, 'backpack')] caption : A double decker bus with neon lights at night.
[(142, 'backpack')] capt. noun : ['bus', 'decker', 'light', 'night']
[(142, 'backpack')] segm. obj  : ['ba

In [15]:
for obj_id, word_count_obj in word_counts.items():
    print(category_id_map[int(obj_id)], word_count_obj.most_common()[:20])
    print()

tie [('man', 305), ('tie', 144), ('woman', 82), ('suit', 70), ('people', 62), ('men', 60), ('picture', 54), ('group', 46), ('shirt', 45), ('cake', 35), ('front', 31), ('couple', 30), ('photo', 26), ('table', 26), ('bus', 24), ('hat', 23), ('girl', 23), ('hand', 22), ('child', 22), ('glass', 22)]

umbrella [('umbrella', 337), ('people', 177), ('woman', 146), ('man', 98), ('street', 76), ('beach', 67), ('person', 54), ('group', 49), ('table', 44), ('girl', 42), ('rain', 35), ('chair', 34), ('city', 30), ('dog', 26), ('sidewalk', 26), ('couple', 24), ('boat', 20), ('day', 19), ('men', 18), ('food', 18)]

handbag [('people', 197), ('woman', 162), ('street', 108), ('man', 100), ('group', 82), ('train', 80), ('bench', 61), ('city', 46), ('person', 44), ('sidewalk', 39), ('bus', 37), ('sign', 34), ('dog', 32), ('phone', 32), ('table', 30), ('building', 28), ('front', 27), ('motorcycle', 26), ('sheep', 22), ('road', 22)]

backpack [('people', 282), ('man', 145), ('group', 107), ('woman', 102),

### Add a flag to annotations in which the secondary object appears in the caption
Based on most common nouns above, create a hardcoded list of nouns to filter captions


In [16]:
# [('tie', '32'), ('umbrella', '28'), ('handbag', '31'), ('backpack', '27')]
common_nouns = {
    '1': ['man', 'men', 'woman', 'women', 'person', 'girl', 'boy', 'couple', 'people'],
    '27': ['backpack', 'luggage'],
    '28': ['umbrella', 'rain'],
    '31': ['luggage', 'handbag', 'bag'],
    '32': ['tie', 'suit'],
}

# common_nouns = {
#     '1': ['man', 'men', 'woman', 'women', 'person', 'girl', 'boy', 'couple', 'people'],
#     '59': ['pizza', 'food'],
#     '61': ['cake', 'food'],
# }

In [17]:
for obj_class in filtered_task1_data:
    for idx, ann in enumerate(filtered_task1_data[obj_class]):
        ann['annotations']['objects_in_caption'] = False
        # find if both objects are in the caption
        for main_obj_noun in common_nouns['1']:
            if main_obj_noun in ann['annotations']["caption_nouns"]:                
                for second_obj_noun in common_nouns[obj_class]:
                    if second_obj_noun in ann['annotations']["caption_nouns"]:
                        ann['annotations']['objects_in_caption'] = True
                        break

In [18]:
# for obj_class in filtered_task1_data:
#     for idx, ann in enumerate(filtered_task1_data[obj_class]):
#         ann['annotations']['objects_in_caption'] = False
#         # find if second object is in the caption
#         for second_obj_noun in common_nouns[obj_class]:
#             if second_obj_noun in ann['annotations']["caption_nouns"]:
#                 ann['annotations']['objects_in_caption'] = True
#                 break

In [19]:
for obj_class, annotations in filtered_task1_data.items():
    print(f"{category_id_map[int(obj_class)]}:  {len([ann for ann in annotations if ann['annotations']['objects_in_caption']])}")

tie:  71
umbrella:  93
handbag:  14
backpack:  19


In [166]:
for obj_class, annotations in filtered_task1_data.items():
    not_in_caption = [ann['annotations']['captions'] for ann in annotations if not ann['annotations']['objects_in_caption']]
    print(f"{category_id_map[int(obj_class)]}:  {not_in_caption[:2]}")

cake:  [['A woman prepares to pour batter into lined muffin tins.', 'A woman making cupcakes in a large kitchen ', 'A woman in an industrial kitchen mixing a batter.', 'there is a woman cooking in a very large kitchen', 'A lady s in the kitchen preparing some kind of meal. '], ['The cake has two motorcycles on the icing. ', 'A birthday cake with two motorcycles on a frosting road', 'The birthday cake has a road with motorcylces in the design.', 'Hands putting motorcycle models onto a birthday cake.', 'Yellow green and white cake with ornaments sitting in a box. ']]
pizza:  [['Two white plates sit on the edge of a table holding pizza slices with fried eggs on them.', 'a close up of two slices of pizza on two plates', 'A wooden table with slices of pizza on it.', 'on this table there are two plates of pizza', 'A wooden rolling pin on a table nears some pizza topped with eggs.'], ['A guy and some friends standing around with longboards. ', 'A young man is standing with his long board.', '

In [168]:
# save processed dataset in a json file
processed_filename = f'filtered_datasets/{data_split}/{dataset_filename[:-5]}_caption.json'
print(f"Processed file saved in {processed_filename}")
with open(processed_filename, "w") as f:
    json.dump(filtered_task1_data, f)

Processed file saved in filtered_datasets/train2017/task5_person_data_no_overlap_caption.json


In [167]:
[ann['annotations']['distinct_obj'] for ann in filtered_task1_data['61'] if 1 not in ann['annotations']['distinct_obj']]

[]

In [160]:
remove_idx = []
for idx, ann in enumerate(filtered_task1_data['61']): 
    if 1 not in ann['annotations']['distinct_obj']:
        print(idx, ann['annotations']['distinct_obj'])
        remove_idx.append(idx)
#         ann.pop(idx)

1139 [67, 50, 51, 52, 59, 61]
1140 [67, 51, 57, 59, 61]
1141 [67, 49, 50, 51, 88, 59, 60, 61]
1142 [67, 44, 47, 49, 51, 59, 61]
1143 [59, 67, 44, 61]
1144 [51, 67, 61, 59]
1145 [67, 47, 48, 50, 59, 61]
1146 [67, 46, 47, 49, 59, 61]
1147 [67, 44, 46, 47, 48, 49, 50, 59, 61]
1148 [48, 59, 67, 61]
1149 [59, 67, 51, 61]
1150 [64, 67, 50, 51, 59, 61]


In [161]:
del filtered_task1_data['61'][1139:1151]

In [162]:
[ann['annotations']['distinct_obj'] for ann in filtered_task1_data['61'] if 1 not in ann['annotations']['distinct_obj']]

[]

# Now with filtered dataset with `dinning table` class

In [78]:
# load filtered file
dataset_filename = 'task2_diningtable_objects_data_no_overlap.json'
# dataset_filename = 'task4_dining_table_data_no_overlap.json'
with open(f'filtered_datasets/{data_split}/{dataset_filename}', "r") as f:
    filtered_task2_data = json.load(f)

In [79]:
word_counts = {}
for obj_class in filtered_task2_data:    
    noun_list = []
    for idx, ann in enumerate(filtered_task2_data[obj_class]):
        captions = ann['annotations']['captions']
        ann['annotations']["caption_nouns"] = []
        for caption in captions:
            tokens = nltk.word_tokenize(caption)
            tag = nltk.pos_tag(tokens)
            list_of_nouns = [wnl.lemmatize(token, pos="n").lower() for token, pos_tag in tag if pos_tag in ['NN', 'NNS']]
            ann['annotations']["caption_nouns"].extend(list_of_nouns)
            noun_list.extend(list_of_nouns)
        if random.random() > 0.9:
            print(f"[{idx, category_id_map[int(obj_class)]}] caption : {caption}")           
            print(f"[{idx, category_id_map[int(obj_class)]}] capt. noun : {sorted(ann['annotations']['caption_nouns'])}")
            print(f"[{idx, category_id_map[int(obj_class)]}] segm. obj  : {sorted([category_id_map[obj_id] for obj_id in ann['annotations']['distinct_obj']])}")
    word_counts[obj_class] = collections.Counter(noun_list)

[(5, 'bottle')] caption : A man sits on a stool in a kitchen.
[(5, 'bottle')] capt. noun : ['bench', 'home', 'kitchen', 'kitchen', 'kitchen', 'kitchen', 'man', 'man', 'man', 'man', 'man', 'stool', 'stool', 'stool', 'table', 'wood']
[(5, 'bottle')] segm. obj  : ['bottle', 'chair', 'clock', 'dining table', 'person', 'refrigerator', 'wine glass']
[(14, 'bottle')] caption : a kitchen with a lot of shelf and cupboard space 
[(14, 'bottle')] capt. noun : ['area', 'bar', 'bar', 'bar', 'ceiling', 'chair', 'counter', 'countertop', 'cupboard', 'cupboard', 'dining', 'kitchen', 'kitchen', 'kitchen', 'lot', 'pantry', 'room', 'shelf', 'shelf', 'space', 'stool', 'stool', 'unit', 'view']
[(14, 'bottle')] segm. obj  : ['bottle', 'chair', 'dining table', 'toaster']
[(20, 'bottle')] caption : A kitchen with a window, oven, island and dishwasher. 
[(20, 'bottle')] capt. noun : ['blackboard', 'blackboard', 'center', 'center', 'chalkboard', 'dishwasher', 'island', 'island', 'kitchen', 'kitchen', 'kitchen', 

[(390, 'bottle')] caption : A laptop shows an image of a laptop on its screen. 
[(390, 'bottle')] capt. noun : ['bottle', 'bottle', 'computer', 'computer', 'image', 'laptop', 'laptop', 'laptop', 'laptop', 'screen', 'table', 'table', 'table', 'table', 'top', 'wooden']
[(390, 'bottle')] segm. obj  : ['bottle', 'chair', 'dining table', 'laptop']
[(395, 'bottle')] caption : Two woman in party costumes standing in front of a man at a small bar serving wine.
[(395, 'bottle')] capt. noun : ['bar', 'bar', 'bottle', 'bottle', 'costume', 'costume', 'front', 'gesture', 'glass', 'glass', 'man', 'man', 'man', 'man', 'pair', 'party', 'side', 'table', 'table', 'wine', 'wine', 'woman', 'woman', 'woman', 'woman', 'woman', 'women']
[(395, 'bottle')] segm. obj  : ['bottle', 'chair', 'dining table', 'person', 'wine glass']
[(419, 'bottle')] caption : A group of wine glasses are sitting on the table.
[(419, 'bottle')] capt. noun : ['bottle', 'bunch', 'dining', 'food', 'glass', 'glass', 'glass', 'glass', 'g

[(615, 'bottle')] caption : A dessert with rainbow sprinkles is about to be eaten
[(615, 'bottle')] capt. noun : ['bottle', 'bottle', 'cake', 'cake', 'dessert', 'milk', 'milk', 'milk', 'pices', 'piece', 'plate', 'plate', 'plate', 'plate', 'rainbow', 'sprinkle', 'straw', 'table']
[(615, 'bottle')] segm. obj  : ['bottle', 'cake', 'dining table']
[(620, 'bottle')] caption : A man in a wine shop carrying a metal suitcase.
[(620, 'bottle')] capt. noun : ['briefcase', 'briefcase', 'case', 'choice', 'counter', 'man', 'man', 'man', 'man', 'man', 'metal', 'shop', 'store', 'suit', 'suitcase', 'suitcase', 'wine', 'wine', 'wine']
[(620, 'bottle')] segm. obj  : ['bottle', 'dining table', 'person', 'suitcase']
[(633, 'bottle')] caption : A group of people playing video games at a banquet
[(633, 'bottle')] capt. noun : ['banquet', 'dining', 'front', 'game', 'game', 'group', 'group', 'line', 'people', 'people', 'people', 'people', 'people', 'remote', 'room', 'room', 'table', 'video', 'wii']
[(633, 'bo

[(902, 'bottle')] caption : a person is trying to feed a bun to a Christmas gnome. 
[(902, 'bottle')] capt. noun : ['bread', 'bread', 'bun', 'bun', 'dog', 'figurine', 'garden', 'gnome', 'gnome', 'gnome', 'gnome', 'hand', 'person', 'person', 'piece', 'piece', 'someone', 'statue']
[(902, 'bottle')] segm. obj  : ['bottle', 'bowl', 'dining table', 'person', 'sandwich']
[(911, 'bottle')] caption : a sandwich, vegetables, and a beer sitting on a table
[(911, 'bottle')] capt. noun : ['bean', 'beer', 'beer', 'bottle', 'broccoli', 'broccoli', 'chili', 'hotdog', 'meat', 'plate', 'plate', 'plate', 'plate', 'roll', 'sandwich', 'sandwich', 'sandwich', 'table', 'table', 'top', 'vegetable', 'vegetable']
[(911, 'bottle')] segm. obj  : ['bottle', 'bowl', 'broccoli', 'dining table', 'sandwich']
[(917, 'bottle')] caption : People sitting around a table with various types of food on it. 
[(917, 'bottle')] capt. noun : ['bowl', 'bowl', 'food', 'food', 'food', 'food', 'food', 'jar', 'people', 'plate', 'plat

[(1133, 'bottle')] caption : A restaurant dining room with tables, chairs, and place settings.
[(1133, 'bottle')] capt. noun : ['ceiling', 'chair', 'chair', 'chair', 'chalkboard', 'clock', 'clock', 'dining', 'dining', 'inside', 'place', 'restaurant', 'restaurant', 'restaurant', 'restaurant', 'room', 'room', 'setting', 'shelf', 'table', 'table', 'table', 'table', 'view']
[(1133, 'bottle')] segm. obj  : ['bottle', 'chair', 'clock', 'cup', 'dining table', 'potted plant', 'wine glass']
[(1136, 'bottle')] caption : A group of people are watching television in a bar.
[(1136, 'bottle')] capt. noun : ['bar', 'bunch', 'game', 'game', 'game', 'game', 'game', 'group', 'guy', 'guy', 'men', 'others', 'people', 'people', 'screen', 'system', 'television', 'video', 'video', 'video', 'wii']
[(1136, 'bottle')] segm. obj  : ['bottle', 'chair', 'cup', 'dining table', 'person', 'potted plant', 'remote', 'tv']
[(1139, 'bottle')] caption : a man is standing over a birthday cake

[(1139, 'bottle')] capt. noun

...

[(1468, 'cup')] caption : A woman sits with her hand between her legs. 
[(1468, 'cup')] capt. noun : ['background', 'doughnut', 'finger', 'hand', 'hand', 'hand', 'knee', 'lap', 'leg', 'leg', 'need', 'person', 'table', 'woman', 'woman', 'woman', 'woman']
[(1468, 'cup')] segm. obj  : ['cell phone', 'cup', 'dining table', 'donut', 'person']
[(1470, 'cup')] caption : Non-fried doughnuts and drinks are for sale in an Asian storefront.
[(1470, 'cup')] capt. noun : ['bakery', 'counter', 'display', 'donut', 'donut', 'doughnut', 'doughnut', 'drink', 'front', 'plate', 'restaurant', 'sale', 'sale', 'sign', 'signage', 'storefront', 'table', 'tray']
[(1470, 'cup')] segm. obj  : ['cake', 'cup', 'dining table', 'donut']
[(1476, 'cup')] caption : Restaurant table with a plate of food, drink and condiment.
[(1476, 'cup')] capt. noun : ['assortment', 'background', 'breakfast', 'breakfast', 'condiment', 'contains', 'drink', 'food', 'food', 'food', 'food', 'fruit', 'fruit', 'glass', 'juice', 'muffin', 'mu

[(1733, 'cup')] caption : Two eggs and two slices of bread on a white plate. 
[(1733, 'cup')] capt. noun : ['bread', 'bread', 'bread', 'bread', 'egg', 'egg', 'egg', 'egg', 'egg', 'piece', 'piece', 'plate', 'plate', 'plate', 'plate', 'plate', 'sit', 'slice', 'table', 'wood']
[(1733, 'cup')] segm. obj  : ['bottle', 'cup', 'dining table', 'knife', 'person', 'spoon']
[(1744, 'cup')] caption : A brown wooden table filled with lots of people sitting at it.
[(1744, 'cup')] capt. noun : ['adult', 'camera', 'camera', 'dinner', 'group', 'group', 'lot', 'people', 'people', 'people', 'people', 'picture', 'posing', 'table', 'table', 'table', 'table', 'wooden']
[(1744, 'cup')] segm. obj  : ['bottle', 'cell phone', 'chair', 'cup', 'dining table', 'fork', 'knife', 'person', 'spoon']
[(1753, 'cup')] caption : A large wooden framed mirror reflecting people eating food.
[(1753, 'cup')] capt. noun : ['choice', 'couple', 'dinner', 'eaterie', 'food', 'food', 'food', 'group', 'people', 'people', 'people', 'p

[(1979, 'cup')] caption : Someone is just about to finish their last bite of a wrap. 
[(1979, 'cup')] capt. noun : ['beer', 'bite', 'dinner', 'dish', 'disk', 'finger', 'food', 'food', 'food', 'food', 'glass', 'half', 'hand', 'hand', 'picture', 'piece', 'plate', 'plate', 'sauce', 'sauce', 'someone', 'time', 'wrap']
[(1979, 'cup')] segm. obj  : ['bottle', 'bowl', 'cup', 'dining table', 'fork', 'person', 'wine glass']
[(1989, 'cup')] caption : On the table is red wine and a large salad.
[(1989, 'cup')] capt. noun : ['cabinet', 'front', 'front', 'glass', 'glass', 'glass', 'laugh', 'man', 'salad', 'something', 'table', 'table', 'table', 'table', 'wine', 'wine', 'wine', 'woman', 'woman', 'woman', 'woman']
[(1989, 'cup')] segm. obj  : ['bottle', 'bowl', 'broccoli', 'chair', 'cup', 'dining table', 'fork', 'person', 'refrigerator', 'sink', 'spoon', 'wine glass']
[(2002, 'cup')] caption : Young boys in a living room have a coffee table full of red cups.
[(2002, 'cup')] capt. noun : ['beer', 'cof

[(2272, 'cup')] caption : Two plates with food sitting on a table
[(2272, 'cup')] capt. noun : ['breakfast', 'food', 'food', 'food', 'food', 'food', 'plate', 'plate', 'plate', 'plate', 'plates', 'restaurant', 'silverware', 'table', 'table', 'table', 'top']
[(2272, 'cup')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork', 'knife', 'spoon']
[(2274, 'cup')] caption : A bowl of grains on a table with bananas.
[(2274, 'cup')] capt. noun : ['apple', 'apple', 'arrangement', 'banana', 'banana', 'banana', 'beverage', 'bowl', 'bowl', 'breakfast', 'cereal', 'crockery', 'food', 'fruit', 'fruit', 'grain', 'orange', 'orange', 'table', 'table', 'tray', 'utensil']
[(2274, 'cup')] segm. obj  : ['apple', 'banana', 'bowl', 'cup', 'dining table', 'knife', 'orange', 'spoon']
[(2299, 'cup')] caption : A child sitting a table with a pizza in front of him.
[(2299, 'cup')] capt. noun : ['boy', 'child', 'child', 'food', 'front', 'front', 'front', 'front', 'kid', 'kid', 'pizza', 'pizza', 'pizza', 'pizza', 'pl

[(2505, 'cup')] caption : A cup of black coffee, biscuits, a piece of meat and grits on a table.
[(2505, 'cup')] capt. noun : ['..', 'biscuit', 'biscuit', 'biscuit', 'breakfast', 'chicken', 'chicken', 'coffee', 'coffee', 'coffee', 'coffee', 'course', 'cup', 'cup', 'cup', 'food', 'grit', 'grit', 'meal', 'meat', 'piece', 'plate', 'plates', 'plates', 'potato', 'potato', 'table', 'table', 'table', 'table', 'top']
[(2505, 'cup')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork', 'knife', 'spoon']
[(2517, 'cup')] caption : A basket of bread and a cup of coffee sit on a table that sits on a deck that looks out to sea.
[(2517, 'cup')] capt. noun : ['basket', 'basket', 'bread', 'bread', 'bread', 'coffee', 'coffee', 'coffee', 'cup', 'cup', 'deck', 'food', 'patio', 'sea', 'sea', 'side', 'sit', 'table', 'table', 'table', 'table', 'table', 'toast', 'waterfront']
[(2517, 'cup')] segm. obj  : ['boat', 'bowl', 'cup', 'dining table', 'knife', 'potted plant', 'spoon', 'umbrella']
[(2522, 'cup')] capt

In [80]:
for obj_id, word_count_obj in word_counts.items():
    print(category_id_map[int(obj_id)], word_count_obj.most_common()[:20])

bottle [('table', 3059), ('people', 1183), ('man', 1017), ('plate', 944), ('pizza', 938), ('woman', 879), ('kitchen', 826), ('wine', 795), ('glass', 777), ('room', 766), ('bottle', 731), ('food', 727), ('group', 562), ('cake', 396), ('sandwich', 333), ('chair', 321), ('front', 287), ('dog', 287), ('top', 285), ('living', 285)]
bowl [('table', 2977), ('plate', 1933), ('food', 1850), ('bowl', 1766), ('room', 579), ('kitchen', 510), ('pizza', 502), ('vegetable', 487), ('woman', 473), ('fruit', 428), ('sandwich', 427), ('man', 411), ('people', 410), ('cake', 379), ('top', 313), ('glass', 310), ('broccoli', 308), ('dish', 292), ('chair', 285), ('living', 281)]
knife [('table', 3444), ('plate', 2755), ('cake', 2033), ('pizza', 1735), ('food', 1510), ('woman', 1063), ('man', 1032), ('people', 971), ('knife', 685), ('sandwich', 551), ('kitchen', 528), ('group', 512), ('glass', 426), ('top', 409), ('slice', 366), ('front', 353), ('restaurant', 352), ('piece', 338), ('person', 327), ('meal', 319

In [81]:
[{key:category_id_map[int(key)]} for key in filtered_task2_data.keys()]

[{'44': 'bottle'}, {'51': 'bowl'}, {'49': 'knife'}, {'47': 'cup'}]

In [82]:
common_nouns = {
    '67': ['table', 'counter'],
    '44': ['wine', 'bottle'],
    '51': ['plate', 'bowl'],
    '49': ['knife'],
    '47': ['glass', 'drink', 'cup'],
}

# common_nouns = {
#     '67': ['table', 'counter'],
#     '59': ['pizza', 'food'],
#     '61': ['cake', 'food'],
# }

In [83]:
for obj_class in filtered_task2_data:
    for idx, ann in enumerate(filtered_task2_data[obj_class]):
        ann['annotations']['objects_in_caption'] = False
        # find if both objects are in the caption
        for noun in common_nouns['67']:
            if noun in ann['annotations']["caption_nouns"]:                
                for noun in common_nouns[obj_class]:
                    if noun in ann['annotations']["caption_nouns"]:
                        ann['annotations']['objects_in_caption'] = True
                        break

In [84]:
for obj_class, annotations in filtered_task2_data.items():
    print(f"{category_id_map[int(obj_class)]}:  {len([ann for ann in annotations if ann['annotations']['objects_in_caption']])}")

bottle:  397
bowl:  805
knife:  225
cup:  825


In [85]:
for obj_class, annotations in filtered_task2_data.items():
    not_in_caption = [ann['annotations']['captions'] for ann in annotations if ann['annotations']['objects_in_caption']]
    print(f"{category_id_map[int(obj_class)]}:  {not_in_caption[:3]}")

bottle:  [['a table with a white plate a sandwich and a bottle', 'A close-up of a plate of food with tortilla chips and a sandwich.', 'A plate on a desk has a partially eaten sandwich and some chips.', 'A sandwich and blue corn chips on a plate.', 'Plated sandwich and chips and bottled tea on a desk.'], ['A man and woman are sitting at the table', 'A man and woman sitting next to each other at a table.', 'An elderly man seated at a bar next to a woman.', 'A man and a woman sit at a table with a bottle of wine.', 'There are two people sitting at the table.'], ['A kitchen scene looking over the table at the refrigerator.', 'A long kitchen table with bottles sitting on the end. ', 'A rectangle shaped wooden dining table with chairs on the sides and several items on the table.', 'Items are laying on a long table in a narrow kitchen.', 'A long wooden table and chairs with a kitchen in the background.']]
bowl:  [['A kitchen view of a dining table with a bowl with bananas.', 'a well maintaine

In [86]:
# save processed dataset in a json file
processed_filename = f'filtered_datasets/{data_split}/{dataset_filename[:-5]}_caption.json'
print(f"Processed file saved in {processed_filename}")
with open(processed_filename, "w") as f:
    json.dump(filtered_task2_data, f)    

Processed file saved in filtered_datasets/train2017/task2_diningtable_objects_data_no_overlap_caption.json


## show stats of each dataset after filter

In [33]:
# # load processed file
# data_split = "train2017"
# with open(f'filtered_datasets/{data_split}/task1_person_accessory_data_no_overlap.json', "r") as f:
#     filtered_task1_data = json.load(f)
    
with open(f'filtered_datasets/{data_split}/task1_person_accessory_data_no_overlap_caption.json', "r") as f:
    filtered_task1_data_caption = json.load(f)    

In [34]:
total_filter = 0
total_no_filter = 0
for object_id, annotations in filtered_task1_data_caption.items():
    with_filter = len([ann['annotations']['captions'] for ann in annotations if ann['annotations']['objects_in_caption']])
    total_filter += with_filter
    total_no_filter += len(annotations)
#     without_filter = len([ann['annotations']['captions'] for ann in annotations if not ann['annotations']['objects_in_caption']])
#     print(f"Class {category_id_map[int(object_id)]} - without filter {len(annotations)} | WITH filter {with_filter}")
    print(f"{with_filter}")
print()    
print(total_filter)
print(total_no_filter, total_filter/total_no_filter)

2121
159
1885
327

4492
14829 0.3029199541439072


In [35]:
with open(f'filtered_datasets/{data_split}/task2_diningtable_objects_data_no_overlap_caption.json', "r") as f:
    filtered_task2_data_caption = json.load(f)  

In [36]:
total_filter = 0
total_no_filter = 0
for object_id, annotations in filtered_task2_data_caption.items():
    with_filter = len([ann['annotations']['captions'] for ann in annotations if ann['annotations']['objects_in_caption']])
    total_filter += with_filter
    total_no_filter += len(annotations)
#     without_filter = len([ann['annotations']['captions'] for ann in annotations if not ann['annotations']['objects_in_caption']])
#     print(f"Class {category_id_map[int(object_id)]} - without filter {len(annotations)} | WITH filter {with_filter}")
print()    
print(total_filter)
print(total_no_filter)
print(total_no_filter, total_filter/total_no_filter)


2252
7933
7933 0.2838774738434388


In [28]:
# # load processed file
# data_split = "train2017"
# with open(f'filtered_datasets/{data_split}/task1_person_accessory_data_no_overlap.json', "r") as f:
#     filtered_task1_data = json.load(f)
    
with open(f'filtered_datasets/{data_split}/task5_person_data_no_overlap_caption.json', "r") as f:
    filtered_task5_data_caption = json.load(f)    

In [37]:
total_filter = 0
total_no_filter = 0
for object_id, annotations in filtered_task5_data_caption.items():
    with_filter = len([ann['annotations']['captions'] for ann in annotations if ann['annotations']['objects_in_caption']])
    total_filter += with_filter
    total_no_filter += len(annotations)
#     without_filter = len([ann['annotations']['captions'] for ann in annotations if not ann['annotations']['objects_in_caption']])
    print(f"Class {category_id_map[int(object_id)]} - without filter {len(annotations)} | WITH filter {with_filter}")

Class cake - without filter 1157 | WITH filter 926
Class pizza - without filter 1189 | WITH filter 960


In [38]:
with open(f'filtered_datasets/{data_split}/task4_dining_table_data_no_overlap_caption.json', "r") as f:
    filtered_task4_data_caption = json.load(f)  

In [39]:
for object_id, annotations in filtered_task4_data_caption.items():
    with_filter = len([ann['annotations']['captions'] for ann in annotations if ann['annotations']['objects_in_caption']])
    total_filter += with_filter
    total_no_filter += len(annotations)
#     without_filter = len([ann['annotations']['captions'] for ann in annotations if not ann['annotations']['objects_in_caption']])
    print(f"Class {category_id_map[int(object_id)]} - without filter {len(annotations)} | WITH filter {with_filter}")

Class cake - without filter 1054 | WITH filter 499
Class pizza - without filter 1090 | WITH filter 746


In [40]:
print()    
print(total_filter)
print(total_no_filter, total_filter/total_no_filter)



3131
4490 0.6973273942093541
